In [1]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
import gensim.models.keyedvectors as word2vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
content='''Dear local newspaper, I've heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren't, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That's amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population's hand-eye coordination would increase, and who wouldn't want thats. Have you ever heard of @LOCATION1? It is a country in @LOCATION2 that no one really knows much about. By looking it up on the computer, you can become an expert on the country! The computer gives us access to a lot of knowledge, and will make us more of people around the world. This can improve our travel. Instead of going to the same place every vacation, you can go to a place you saw online that looks just as exciting. Also, by learning about other countries and the people in them, we gain respect for them. We @MONTH1 also learn about their traditions, and culture. "The internet gives us access to the world," says @PERSON1, "we can learn so many new things that will make us more of people, countries, and the environment." @CAPS1 is right, we can also be aware of the environment. Since global warming is a problem, we can learn how o preserve foreign places by learning about them on the internet. With a click of a mouse you can virtually have the whose world at your fingertips! In a recent study @PERCENT2 of people said they don't have time to meet with old friends. The internet, however has made it quick and easy to talk to people online. Websites like @CAPS2, @CAPS3, and @CAPS4 can keep your social life in tact, and can even reunite yu with old friends. For example, @CAPS2 suggest family. You can keep in touch wih many people trough @CAPS2! Also, you can make new friends. Most teens say they are always willing to meet new people, and the internet is a great way to do so! Sites like @CAPS7, and @CAPS8 can can help you find friends by connecting you in a chat room to a random person, just to talk. I once talked with someone on @CAPS7 for over an hour! We found that we both have @CAPS4 accounts and keep in touch there! This also helps you gain confidence because you @MONTH1 have the courage to say something over the computer that you would't say real life. The computer can really amp up your social life. As you can see. computers don't just "your barain," they imprdove hand-eye coordination, help you learn about far away places, and improve your social life by allowing you to talk to others. Computers benefit the society more than anymore will ever know!'''

In [3]:
content

'Dear local newspaper, I\'ve heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren\'t, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That\'s amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population\'s hand-eye coordination would increase,

In [4]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

In [5]:
def essay_to_wordlist(essay_v):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

In [6]:
def essay_to_sentences(essay_v):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence))
    return sentences

In [7]:
def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])       
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [8]:
def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [9]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

In [10]:
def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [15]:
import math
if len(content) > 20:
                num_features = 300
                model = word2vec.KeyedVectors.load_word2vec_format('word2vecmodel.bin', binary=True)
                clean_test_essays = []
                clean_test_essays.append(essay_to_wordlist( content))
                testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
                testDataVecs = np.array(testDataVecs)
                testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

                lstm_model = get_model()
                lstm_model.load_weights('final_lstm.h5')
                preds = lstm_model.predict(testDataVecs)
                preds = np.around(preds)



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 300)            721200    
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [16]:
preds

array([[9.]], dtype=float32)